<a href="https://colab.research.google.com/github/midway-official/DNS_solver/blob/main/DNS_PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install  tensorflow
!pip install  numpy

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # 导入tqdm
from concurrent.futures import ThreadPoolExecutor

In [3]:

def load_data(folder):
    """
    从指定文件夹加载 u, v, p, x, y 的数据。
    """
    u = np.loadtxt(os.path.join(folder, "u.dat"))
    v = np.loadtxt(os.path.join(folder, "v.dat"))
    p = np.loadtxt(os.path.join(folder, "p.dat"))
    x = np.loadtxt(os.path.join(folder, "x.dat"))
    y = np.loadtxt(os.path.join(folder, "y.dat"))
    return u, v, p, x, y

def process_data(u, v, p, x, y):
    """
    将 u, v, p 矩阵与坐标 (x, y) 对应，生成 PINN 的输入数据 (x, y, t) 和输出 (u, v, p)。
    """
    rows, cols = u.shape
    xyz_train = []
    uvp_train = []

    for i in range(rows):
        for j in range(cols):
            # 矩阵中的 (i, j) 转换为坐标 (x[j], y[-(i+1)])
            x_coord = x[j]
            y_coord = y[-(i+1)]  # y 倒序索引
            xyz_train.append([x_coord, y_coord])  # 加入时间维度时可扩展
            uvp_train.append([u[i, j], v[i, j], p[i, j]])

    return np.array(xyz_train), np.array(uvp_train)



In [4]:
"""
def load_multiple_timesteps(base_folder, timesteps):

    xyz_train_all = []
    uvp_train_all = []
    dt = 0.01
    for t in timesteps:
        print(f"Loading: t = {t}")
        folder = os.path.join(base_folder, str(t))  # 时间步文件夹
        u, v, p, x, y = load_data(folder)
        xyz_train, uvp_train = process_data(u, v, p, x, y)

        # 为每个点添加时间信息 t
        t_column = np.full((xyz_train.shape[0], 1), dt*t)
        xyz_train = np.hstack([xyz_train, t_column])

        # 合并所有时间步的数据
        xyz_train_all.append(xyz_train)
        uvp_train_all.append(uvp_train)

    return np.vstack(xyz_train_all), np.vstack(uvp_train_all)
"""

'\ndef load_multiple_timesteps(base_folder, timesteps):\n\n    xyz_train_all = []\n    uvp_train_all = []\n    dt = 0.01\n    for t in timesteps:\n        print(f"Loading: t = {t}")\n        folder = os.path.join(base_folder, str(t))  # 时间步文件夹\n        u, v, p, x, y = load_data(folder)\n        xyz_train, uvp_train = process_data(u, v, p, x, y)\n\n        # 为每个点添加时间信息 t\n        t_column = np.full((xyz_train.shape[0], 1), dt*t)\n        xyz_train = np.hstack([xyz_train, t_column])\n\n        # 合并所有时间步的数据\n        xyz_train_all.append(xyz_train)\n        uvp_train_all.append(uvp_train)\n\n    return np.vstack(xyz_train_all), np.vstack(uvp_train_all)\n'

In [5]:

def load_multiple_timesteps(base_folder, timesteps):
    """
    高效加载多个时间步的数据，并添加时间信息。
    使用并行处理加速数据加载，并显示进度条。
    """
    dt = 0.01
    xyz_train_all = []
    uvp_train_all = []

    def load_and_process_timestep(t):
        """
        加载和处理单个时间步的数据
        """
        folder = os.path.join(base_folder, str(t))  # 时间步文件夹
        u, v, p, x, y = load_data(folder)
        xyz_train, uvp_train = process_data(u, v, p, x, y)

        # 为每个点添加时间信息 t
        t_column = np.full((xyz_train.shape[0], 1), dt*t)
        xyz_train = np.hstack([xyz_train, t_column])

        return xyz_train, uvp_train

    # 使用 ThreadPoolExecutor 来并行处理多个时间步，并显示进度条
    with ThreadPoolExecutor() as executor:
        # 使用 tqdm 显示进度条
        results = list(tqdm(executor.map(load_and_process_timestep, timesteps), total=len(timesteps)))

    # 将所有时间步的数据合并
    for xyz_train, uvp_train in results:
        xyz_train_all.append(xyz_train)
        uvp_train_all.append(uvp_train)

    # 合并所有时间步的数据
    return np.vstack(xyz_train_all), np.vstack(uvp_train_all)

In [6]:
# 加载时间步 0 到 800 的数据
result_folder = '/content/drive/MyDrive/result30'  # 结果文件夹路径
timesteps = range(0, 300)  # 200 个时间步

# 加载数据
xyz_data_all, uvp_data_all = load_multiple_timesteps(result_folder, timesteps)

100%|██████████| 300/300 [00:07<00:00, 40.84it/s]


In [7]:
#打印检查
print("xyz_data_all shape:", xyz_data_all.shape)  # 打印输入的形状
print("uvp_data_all shape:", uvp_data_all.shape)  # 打印输出的形状

xyz_data_all shape: (307200, 3)
uvp_data_all shape: (307200, 3)


In [8]:
# 保存为 .npy 文件
np.save('xyz_data_all.npy', xyz_data_all)
np.save('uvp_data_all.npy', uvp_data_all)

In [9]:
# 定义 PINN 模型
class PINN(tf.keras.Model):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.hidden_layers = []
        for width in layers[1:-1]:
            self.hidden_layers.append(tf.keras.layers.Dense(width, activation='tanh'))
        self.output_layer = tf.keras.layers.Dense(layers[-1])

    def call(self, x):
        for layer in self.hidden_layers:
            x = layer(x)
        return self.output_layer(x)


In [10]:

# 构建 PINN 模型
layers = [3,64,64,64,64,64,64,64, 3]  # 输入3个（x, y, t），输出3个（u, v, p）
nu = 1/10000  # 粘性系数
pinn = PINN(layers)

In [11]:

# 计算瞬态 Navier-Stokes 方程的损失
def compute_loss_with_data(xyz_train, uvp_train, pinn, nu):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([xyz_train])

        # 预测 u, v, p
        outputs = pinn(xyz_train)
        u_pred, v_pred, p_pred = outputs[:, 0:1], outputs[:, 1:2], outputs[:, 2:3]

        # 数据驱动的损失
        data_loss = tf.reduce_mean(tf.square(tf.cast(u_pred, tf.float32) - tf.cast(uvp_train[:, 0:1], tf.float32))) + \
            tf.reduce_mean(tf.square(tf.cast(v_pred, tf.float32) - tf.cast(uvp_train[:, 1:2], tf.float32))) + \
            tf.reduce_mean(tf.square(tf.cast(p_pred, tf.float32) - tf.cast(uvp_train[:, 2:3], tf.float32)))

        # 计算一阶导数
        u_x = tape.gradient(u_pred, xyz_train)[:, 0:1]
        u_y = tape.gradient(u_pred, xyz_train)[:, 1:2]
        u_t = tape.gradient(u_pred, xyz_train)[:, 2:3]
        v_x = tape.gradient(v_pred, xyz_train)[:, 0:1]
        v_y = tape.gradient(v_pred, xyz_train)[:, 1:2]
        v_t = tape.gradient(v_pred, xyz_train)[:, 2:3]
        p_x = tape.gradient(p_pred, xyz_train)[:, 0:1]
        p_y = tape.gradient(p_pred, xyz_train)[:, 1:2]

    # 计算二阶导数
    u_xx = tape.gradient(u_x, xyz_train)[:, 0:1]
    u_yy = tape.gradient(u_y, xyz_train)[:, 1:2]
    v_xx = tape.gradient(v_x, xyz_train)[:, 0:1]
    v_yy = tape.gradient(v_y, xyz_train)[:, 1:2]

    del tape  # 删除梯度带释放资源

    # Navier-Stokes 方程的残差项
    # 动量方程（x方向）
    f_u = tf.cast(u_t, tf.float32) + tf.cast(u_pred, tf.float32) * tf.cast(u_x, tf.float32) + \
          tf.cast(v_pred, tf.float32) * tf.cast(u_y, tf.float32) + tf.cast(p_x, tf.float32) - \
          tf.cast(nu, tf.float32) * (tf.cast(u_xx, tf.float32) + tf.cast(u_yy, tf.float32))

    # 动量方程（y方向）
    f_v = tf.cast(v_t, tf.float32) + tf.cast(u_pred, tf.float32) * tf.cast(v_x, tf.float32) + \
          tf.cast(v_pred, tf.float32) * tf.cast(v_y, tf.float32) + tf.cast(p_y, tf.float32) - \
          tf.cast(nu, tf.float32) * (tf.cast(v_xx, tf.float32) + tf.cast(v_yy, tf.float32))

    # 连续性方程
    f_continuity = tf.cast(u_x, tf.float32) + tf.cast(v_y, tf.float32)

    # 计算 Navier-Stokes 方程残差损失
    ns_loss = tf.reduce_mean(tf.square(f_u)) + tf.reduce_mean(tf.square(f_v)) + tf.reduce_mean(tf.square(f_continuity))

    # 总损失
    total_loss = data_loss + ns_loss
    return total_loss

In [12]:

# 定义优化器和训练过程
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def train_step(xyz_train, uvp_train, pinn, optimizer, nu):
    with tf.GradientTape() as tape:
        loss = compute_loss_with_data(xyz_train, uvp_train, pinn, nu)
    gradients = tape.gradient(loss, pinn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, pinn.trainable_variables))
    return loss

In [13]:

def split_data(xyz_data_all, uvp_data_all, train_ratio=0.8, val_ratio=0.1):
    """
    将数据集划分为训练集、验证集和测试集。

    参数:
        xyz_data_all (np.ndarray): 包含所有坐标和时间数据
        uvp_data_all (np.ndarray): 包含所有速度和压力数据
        train_ratio (float): 训练集的比例
        val_ratio (float): 验证集的比例

    返回:
        (train, validation, test): 训练集、验证集、测试集的元组
    """
    total_size = xyz_data_all.shape[0]  # 使用 xyz_data_all，而不是 xyz_data
    indices = np.arange(total_size)
    np.random.shuffle(indices)

    train_size = int(total_size * train_ratio)
    val_size = int(total_size * val_ratio)
    test_size = total_size - train_size - val_size

    # 切分数据
    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size + val_size]
    test_indices = indices[train_size + val_size:]

    xyz_train = xyz_data_all[train_indices]
    uvp_train = uvp_data_all[train_indices]

    xyz_val = xyz_data_all[val_indices]
    uvp_val = uvp_data_all[val_indices]

    xyz_test = xyz_data_all[test_indices]
    uvp_test = uvp_data_all[test_indices]

    return (xyz_train, uvp_train), (xyz_val, uvp_val), (xyz_test, uvp_test)

In [14]:
# 数据加载
(xyz_train, uvp_train), (xyz_val, uvp_val), (xyz_test, uvp_test) = split_data(xyz_data_all, uvp_data_all)

print(f"Training data shape: {xyz_train.shape}, {uvp_train.shape}")
print(f"Validation data shape: {xyz_val.shape}, {uvp_val.shape}")
print(f"Test data shape: {xyz_test.shape}, {uvp_test.shape}")


Training data shape: (245760, 3), (245760, 3)
Validation data shape: (30720, 3), (30720, 3)
Test data shape: (30720, 3), (30720, 3)


In [1]:
#转化为单精度张量
xyz_val = tf.convert_to_tensor(xyz_val, dtype=tf.float32)
uvp_val = tf.convert_to_tensor(uvp_val, dtype=tf.float32)
epochs = 200


for epoch in tqdm(range(epochs), desc="Training Progress", ncols=100):
    # 训练
    loss = train_step(xyz_train, uvp_train, pinn, optimizer, nu)

    # 每 50 个 epoch 打印训练损失和验证损失
    if epoch % 50 == 0:
        # 验证
        val_loss = tf.reduce_mean(compute_loss_with_data(xyz_val, uvp_val, pinn, nu))  # 验证集损失
        print(f"Epoch {epoch}/{epochs}, Training Loss: {loss.numpy():.6f}, Validation Loss: {val_loss.numpy():.6f}")

NameError: name 'tf' is not defined

In [ ]:

# 保存最终模型
pinn.save("pinn_mode.h5")

In [ ]:
# 计算测试集损失
xyz_test = tf.convert_to_tensor(xyz_test, dtype=tf.float32)
uvp_test = tf.convert_to_tensor(uvp_test, dtype=tf.float32)
test_loss = tf.reduce_mean(compute_loss_with_data(xyz_test, uvp_test, pinn, nu))  # test集损失
print(f"Test Loss: {test_loss.numpy()}")

In [ ]:

# 定义计算网格
x_range = np.linspace(0, 1, 100)  # x方向
y_range = np.linspace(0, 1, 100)  # y方向
X, Y = np.meshgrid(x_range, y_range)

# 定义时刻 t
t = 2  # 假设是 t=0.05 的时刻

# 将网格点打包为输入数据
input_data = np.hstack([X.flatten()[:, None], Y.flatten()[:, None], np.full((X.size, 1), t)])

# 使用 PINN 预测速度和压力
pinn_output = pinn.predict(input_data)
u = pinn_output[:, 0].reshape(X.shape)  # 提取 u 速度分量
v = pinn_output[:, 1].reshape(X.shape)  # 提取 v 速度分量
p = pinn_output[:, 2].reshape(X.shape)  # 提取压力

# 归一化速度
magnitude = np.sqrt(u**2 + v**2)
u_normalized = u / magnitude
v_normalized = v / magnitude

In [ ]:
import matplotlib.pyplot as plt

# 绘制图像
plt.figure(figsize=(15, 5))

# 绘制速度大小的等高线图
plt.subplot(1, 3, 1)
contour = plt.contourf(X, Y, magnitude, levels=20, cmap='viridis')
plt.colorbar(contour)
plt.title('Velocity Magnitude')
plt.xlabel('X')
plt.ylabel('Y')

# 绘制归一化的速度矢量图，叠加压力等高线
plt.subplot(1, 3, 2)
contour_p = plt.contourf(X, Y, p, levels=20, cmap='autumn', alpha=0.6)
plt.colorbar(contour_p)
plt.title('Normalized Velocity and Pressure')
plt.xlabel('X')
plt.ylabel('Y')

# 绘制流线图
plt.subplot(1, 3, 3)
plt.streamplot(X, Y, u, v, color='b', linewidth=0.3, density=3)
plt.title('Streamlines')
plt.xlabel('X')
plt.ylabel('Y')

plt.tight_layout()
plt.show()